In [2]:
import dpkt
import socket

def mac_addr(address):
    """Convert a MAC address to a readable/printable string

       Args:
           address (str): a MAC address in hex form (e.g. '\x01\x02\x03\x04\x05\x06')
       Returns:
           str: Printable/readable MAC address
    """
    return ':'.join('%02x' % ord(b) for b in address)


def inet_to_str(inet):
    """Convert inet object to a string

        Args:
            inet (inet struct): inet network address
        Returns:
            str: Printable/readable IP address
    """
    # First try ipv4 and then ipv6
    try:
        return socket.inet_ntop(socket.AF_INET, inet)
    except ValueError:
        return socket.inet_ntop(socket.AF_INET6, inet)


f = open("pcaps/small_torrent.pcap")
pcap = dpkt.pcap.Reader(f)

In [61]:
# This is very optimistic. Won't work if the string is wrongly formatted
def bdecode(string):
    stack = []
    while (len(string)  > 0):
        if (string[0] == "d"):
            stack.append({})
            string = string[1:]
        elif (string[0] == "l"):
            stack.append([])
            string = string[1:]
        elif (string[0] == "i"):
            splitted = string[1:].split('e', 1)
            stack.append(int(splitted[0]))
            string = splitted[1]
        elif (string[0] == "e"):
            item = stack.pop()
            items = []
            while (len(stack) > 0 and item != {} and item != []):
                items.append(item)
                item = stack.pop()
            items.reverse()
            if (len(items) == 0):
                raise ValueError('empty list or dictionary')
            if (item == []):
                stack.append(items)
            elif (item == {}):
                dic = {}
                if (len(items) % 2 == 1):
                    raise ValueError('odd number of key/values in dictionary')
                while (len(items) != 0):
                    dic[items[0]] = items[1]
                    items = items[2:]
                stack.append(dic)
            string = string[1:]
                
        else:
            splitted = string.split(':', 1)
            str_len = int(splitted[0])
            string = splitted[1]
            stack.append(string[:str_len])
            string = string[str_len:]
        
    return stack[0]

In [60]:
# For each packet in the pcap process the contents
for timestamp, buf in pcap:

    # Print out the timestamp in UTC
    # print 'Timestamp: ', str(datetime.datetime.utcfromtimestamp(timestamp))

    # Unpack the Ethernet frame (mac src/dst, ethertype)
    eth = dpkt.ethernet.Ethernet(buf)
    # print 'Ethernet Frame: ', mac_addr(eth.src), mac_addr(eth.dst), eth.type

    # Make sure the Ethernet frame contains an IP packet
    if not isinstance(eth.data, dpkt.ip.IP):
        # print 'Non IP Packet type not supported %s\n' % eth.data.__class__.__name__
        continue

    # Now unpack the data within the Ethernet frame (the IP packet)
    # Pulling out src, dst, length, fragment info, TTL, and Protocol
    ip = eth.data

    # Pull out fragment information (flags and offset all packed into off field, so use bitmasks)
    do_not_fragment = bool(ip.off & dpkt.ip.IP_DF)
    more_fragments = bool(ip.off & dpkt.ip.IP_MF)
    fragment_offset = ip.off & dpkt.ip.IP_OFFMASK
    pkt = ip.data
    if isinstance(pkt, dpkt.tcp.TCP):
        print "TCP"
    elif isinstance(pkt, dpkt.udp.UDP):
        print "UDP"
    if ("peers" in str(pkt.data)):
        print bdecode(pkt.data)
    try:
        print pkt.sport
    except AttributeError:
        print "error"
    # Print out the info
    print 'IP: %s -> %s   (len=%d ttl=%d DF=%d MF=%d offset=%d)\n' % \
          (inet_to_str(ip.src), inet_to_str(ip.dst), ip.len, ip.ttl, do_not_fragment, more_fragments, fragment_offset)

UDP
49873
IP: 192.168.0.101 -> 239.255.255.250   (len=63 ttl=64 DF=1 MF=0 offset=0)

UDP
137
IP: 192.168.0.102 -> 151.101.1.69   (len=78 ttl=128 DF=0 MF=0 offset=0)

UDP
36360
IP: 192.168.0.102 -> 173.52.246.19   (len=131 ttl=128 DF=0 MF=0 offset=0)

error
IP: 173.52.246.19 -> 192.168.0.102   (len=159 ttl=50 DF=0 MF=0 offset=0)

UDP
137
IP: 192.168.0.102 -> 151.101.1.69   (len=78 ttl=128 DF=0 MF=0 offset=0)

TCP
54662
IP: 192.168.0.102 -> 198.252.206.25   (len=41 ttl=128 DF=1 MF=0 offset=0)

UDP
45816
IP: 192.168.0.101 -> 192.168.0.255   (len=63 ttl=64 DF=1 MF=0 offset=0)

TCP
443
IP: 198.252.206.25 -> 192.168.0.102   (len=52 ttl=51 DF=1 MF=0 offset=0)

UDP
137
IP: 192.168.0.102 -> 151.101.1.69   (len=78 ttl=128 DF=0 MF=0 offset=0)

UDP
1901
IP: 192.168.1.1 -> 239.255.255.250   (len=328 ttl=3 DF=0 MF=0 offset=0)

UDP
1901
IP: 192.168.1.1 -> 239.255.255.250   (len=384 ttl=3 DF=0 MF=0 offset=0)

UDP
1901
IP: 192.168.1.1 -> 239.255.255.250   (len=312 ttl=3 DF=0 MF=0 offset=0)

UDP
1901
IP

TypeError: getattr(): attribute name must be string

SyntaxError: EOL while scanning string literal (<ipython-input-59-7ecbf8fd58bf>, line 1)